In [20]:
import pathlib
from PyPDF2 import PdfReader
import pandas as pd
import re
import polars as pl
import writexlsx as wx

In [21]:

def rm_space(x: str) -> str:
    """remove unexpected space from table strings

    Args:
        x (str): the row string read from pdf

    Returns:
        str: the cleaned row string
    """
    pattern = r"(.+)(?=\s\d{8})"
    match = re.match(pattern, x)
    if match is None:
        return x
    prod_name = match.group(0).replace(" ", "")
    return re.sub(pattern, prod_name, x)


def rm_garbage(x: str) -> list[str]:
    """remove garbage letters

    Args:
        x (str): _description_

    Returns:
        list[str]: _description_
    """
    garbage = ['½ö¹©°²Áª×Ê²ú²Î¿¼', 'NÅO\x9b[\x89\x80T\x8dDN§O\x7fu(']
    for elem in garbage:
        x = x.replace(elem, "")
    return x.strip().split("\n")

def rate2num(x: pd.Series) -> list | pd.Series:
    if not pd.api.types.is_string_dtype(x):
        return x
    out: list = []
    for elem in x:
        if pd.isna(elem) or elem == "-":
            out.append(pd.NA)
        elif elem.find("%") > -1:
            out.append(pd.to_numeric(elem.replace("%", "")) / 100.0)
        else:
            out.append(pd.to_numeric(elem))
    return out


In [22]:
# pdf_path = pathlib.Path("/Users/shrektan/Library/CloudStorage/OneDrive-共享的库-onedrive/安联资管文档/监管和协会资料/组合类产品信息/保险资产管理产品行业报告（2022年12月）-组合行情.pdf")


def to_tbl(x: list[str]) -> pd.DataFrame:
    txt = filter(lambda x: x.count(" ") <=6 and x.count(" ") >=3 and (x.find("公司") != -1), x)
    cols = [
        "序号", "管理机构名称", "产品只数",
        "固定收益类", "混合类", "权益类", "合计"
    ]
    return pd.DataFrame(map(lambda x: x.split(" "), txt), columns=cols)

def read_tbl(pdf_path: pathlib.Path) -> pd.DataFrame:
    """read table from pdf

    Args:
        pdf_path (pathlib.Path): the path to the pdf file

    Returns:
        pd.DataFrame: the table
    """
    p = PdfReader(pdf_path)
    txt = p.pages[1].extract_text()
    clean_txt = rm_garbage(rm_space(txt))
    # print(clean_txt)
    df = to_tbl(clean_txt)
    df["产品只数"] = rate2num(df["产品只数"])
    df["固定收益类"] = rate2num(df["固定收益类"])
    df["混合类"] = rate2num(df["混合类"])
    df["权益类"] = rate2num(df["权益类"])
    df["合计"] = rate2num(df["合计"])
    return df


In [23]:

dates = [
    (2021, 12),
    (2022, 12),
    (2023, 3),
    (2023, 6),
    (2023, 9),
    (2023, 10),
]
out = dict()
for d in dates:
    year = d[0]
    month = d[1]
    nm = f"{year}.{month}"
    path_temp = "/Users/shrektan/Library/CloudStorage/OneDrive-共享的库-onedrive/安联资管文档/监管和协会资料/组合类产品信息/保险资产管理产品行业报告（{year}年{month}月）-组合行情.pdf"
    out[nm] = read_tbl(pathlib.Path(path_temp.format(year=year, month=month)))

wx.write(out, "~/Downloads/out.xlsx", overwrite=True, open=True)
# out = pl.from_pandas(df)
# out.to_csv("out.csv")

/Users/shrektan/Library/Caches/pypoetry/virtualenvs/py-scripts-dIaQrCa2-py3.11/lib/python3.11/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /GBK-EUC-H not implemented yet
  warnings.warn(
/Users/shrektan/Library/Caches/pypoetry/virtualenvs/py-scripts-dIaQrCa2-py3.11/lib/python3.11/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UTF16-H not implemented yet
  warnings.warn(


PosixPath('/Users/shrektan/Downloads/out.xlsx')